In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d gpiosenka/cats-in-the-wild-image-classification

 99% 122M/123M [00:07<00:00, 22.8MB/s]
100% 123M/123M [00:07<00:00, 16.5MB/s]


In [5]:
!unzip -q cats-in-the-wild-image-classification.zip

In [6]:
import tensorflow as tf
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator

In [7]:
# путь к датасету
train_dir = '/content/train'
validation_dir = '/content/valid'

In [20]:
# параметры модели
img_width, img_height = 224, 224
batch_size = 16
epochs = 20
train_samples = 2090
validation_samples = 230

In [9]:
# создание генераторов данных
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                              target_size=(img_width, img_height),
                                                              batch_size=batch_size,
                                                              class_mode='categorical')

Found 2339 images belonging to 10 classes.
Found 50 images belonging to 10 classes.


In [10]:
# загрузка предобученной модели
base_model = VGG16(weights='imagenet',
                   include_top=False,
                   input_shape=(img_width, img_height, 3))

58889256/58889256 [==============================] - 3s 0us/step


In [11]:
# заморозка весов предобученной модели
for layer in base_model.layers:
    layer.trainable = False

In [12]:
# создание новой модели на основе предобученной
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [14]:
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=1e-4),
              metrics=['accuracy'])

In [21]:
# дообучение модели
history = model.fit(train_generator,
                              steps_per_epoch=train_samples // batch_size,
                              epochs=epochs,
                              validation_data=validation_generator,
                              validation_steps=validation_samples // batch_size)

Epoch 1/20
130/130 [==============================] - ETA: 0s - loss: 0.8599 - accuracy: 0.6952

130/130 [==============================] - 28s 215ms/step - loss: 0.8599 - accuracy: 0.6952 - val_loss: 0.5272 - val_accuracy: 0.8000
Epoch 2/20
130/130 [==============================] - 27s 208ms/step - loss: 0.8033 - accuracy: 0.7078
Epoch 3/20
130/130 [==============================] - 28s 213ms/step - loss: 0.7732 - accuracy: 0.7194
Epoch 4/20
130/130 [==============================] - 28s 215ms/step - loss: 0.7276 - accuracy: 0.7447
Epoch 5/20
130/130 [==============================] - 28s 213ms/step - loss: 0.7332 - accuracy: 0.7332
Epoch 6/20
130/130 [==============================] - 28s 213ms/step - loss: 0.7209 - accuracy: 0.7533
Epoch 7/20
130/130 [==============================] - 28s 214ms/step - loss: 0.7236 - accuracy: 0.7470
Epoch 8/20
130/130 [==============================] - 28s 212ms/step - loss: 0.6896 - accuracy: 0.7533
Epoch 9/20
130/130 [==============================] - 27s 208ms/step - loss: 0.6646 - accuracy: 0.7707
Epoch 10/20
130/130 [=====================

In [28]:
test_datagen = ImageDataGenerator(rescale=1./255)

# Загрузка тестовых данных
test_generator = test_datagen.flow_from_directory(
        'test',
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

# Оценка точности модели на тестовых данных
scores = model.evaluate(test_generator)
print("Test Accuracy:", scores[1])

Found 50 images belonging to 10 classes.
4/4 [==============================] - 0s 59ms/step - loss: 0.3338 - accuracy: 0.9200
Test Accuracy: 0.9200000166893005
